##### Saving words from newspaper titles in textfiles

In [25]:
import json

# Path to the JSON file
json_file_path = 'cdx_results_2019.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# List to store the URLs
urls = []

# Loop through the data and extract 'url' values
for entry in data:
    if 'url' in entry:  # Check if 'url' key exists
        urls.append(entry['url'])  # Add the URL to the list

# Open a file to write the results
with open("titles.txt", "w") as file:
    # Loop through each URL
    for url in urls:
        # Remove 'https://' and split the URL at slashes
        parts = url.split('/')
        
        # Extract the last part after the slash, which usually contains the title
        title_with_params = parts[-1]
        
        # Remove any parameters (everything after '?')
        title = title_with_params.split('?')[0]
        
        # Replace hyphens with spaces to improve readability
        formatted_title = title.replace('-', ' ')
        
        # Write the formatted title to the file
        file.write(f"{formatted_title}\n")

print("Titles extracted and written to 'titles.txt'.")

# maybe store urls and words wich belong togethers in a dictionary

Titles extracted and written to 'titles.txt'.


In [26]:
import json

# Function to split the URL into words from the title
def split_url_in_words(url):
    # Remove 'https://' and split the URL at slashes
    parts = url.split('/')
    
    # Extract the last part after the slash, which usually contains the title
    title_with_params = parts[-1]
    
    # Remove any parameters (everything after '?')
    title = title_with_params.split('?')[0]
    
    # Replace hyphens with spaces to improve readability
    formatted_title = title.replace('-', ' ')
    
    # Return the split words
    return formatted_title

# Path to the JSON file
json_file_path = 'cdx_results_2019.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Empty dictionary to store the URLs as keys and the split title words as values
urls = {}

# Iterate through the JSON data
for entry in data:
    if 'url' in entry:
        #url = entry['url']  # Get the full URL
        #split_title = split_url_in_words(url)  # Split the URL title into words
        #urls[str(entry['url'])] = split_title  # Store the URL as key and split title as value
        urls[str(entry['url'])] = split_url_in_words(entry['url'])

# Writing dictionary to a textfile
with open('urls_titles.json', 'w') as output_file:
    json.dump(urls, output_file, indent=4)

 Trying to scrape only the <head> elements in the links and look up in the headline if Chega is written as: 'Chega' or 'CHEGA':

In [32]:
import json
import requests
from bs4 import BeautifulSoup

# Path to the JSON file
json_file_path = 'test_for_scraping_cdx_results.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

titles = {}

# Observation: in cmjornal, jn.pt and eco.sapo.pt there is the heading in the title element. so dont search for h-elements, only for title
# Loop through URLs; scrape every head-container and search for the head element
for v in data:
    # Requesting the website and getting html-content
    response = requests.get(v['url'], timeout=50, stream=True)
    html_content = b""
    for chunk in response.iter_content(chunk_size=512):
        html_content += chunk
    if b"</title>" in html_content:  # Stop when </title> is found
        break

    html_content = response.content

    # HTML parsing with BeautifulSoup
    soup = BeautifulSoup(html_content, "lxml")

    # all title containers are extracted from title
    title_tag = soup.find("title")

    # splitting content out of full tag string

    title_content = title_tag.get_text()

    # check, if title contains chega; checking at 'Chega' and 'CHEGA', because 'chega' like in title has different meaning and matches more often 
    if "Chega" in title_content or "CHEGA" in title_content:
        url = v['url']
        titles[url] = title_content

with open('valid_url_values.txt', 'w') as convert_file: 
     convert_file.write(json.dumps(titles))

In [28]:
# Debug output:

for url, title in titles.items():
    print(f'{url}: {title}')

https://www.cmjornal.pt/politica/amp/andre-ventura-diz-chega-vai-impedir-extrema-direita-em-portugal: André Ventura diz que Chega vai impedir extrema-direita em Portugal - Política - Correio da Manhã
https://www.cmjornal.pt/politica/amp/partido-chega-de-andre-ventura-inicia-formalizacao-para-ser-alternativa-a-direita-que-parece-nao-existir: Partido 'CHEGA' inicia formalização para ser alternativa "à direita que parece não existir"  - Política - Correio da Manhã
